In [1]:
# importing library

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.cross_validation import train_test_split # for spliting dataset
from sklearn.feature_extraction.text import CountVectorizer # bow-->1gram and 2 gram
from sklearn.feature_extraction.text import TfidfVectorizer # tf-idf
from gensim.models import Word2Vec  # w2v
from gensim.models import KeyedVectors # to understanding w2v using google pre trained model
from sklearn.metrics import accuracy_score # to check the accuracy of model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import cross_val_score # k-fold cv

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


>   # Loading the dataset

### direct downloading from Kaggle using curl

In [ ]:
# remove '#' sign to download data-set
#!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3575.0 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8" --header="Accept-Language: en-US,en;q=0.9" "https://storage.googleapis.com/kaggle-datasets/18/2157/Reviews.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1539455264&Signature=sUXQz0kX2395nqKOQppw23fSVbEPjFPhNtI1TC5FN1fOhx8RBGKLP1d1UZgiftoI1XI3TLxlFu6JwY%2BL7LwhxM46VQp89iJ%2BJZ8PaaY%2F61q8y%2BVchuDT8v4UnmbJ5%2Bkvf77HaNiJrAcqSY1K0C66npHhhaAgMzmvHJtTOnpZ70LFbZ6g1X%2Bh%2Ba2Tmkuiae%2F3CVIjnkE7sO2X4l3o5x4H45gu3EWzyrlqWJDwx0EJmDbnsOCsCzIEie7in70HwkDOwxGq9WdCGscFlFVDfd8W%2BV00yUfT82%2F2%2F4H4WGrnvI0uxFECct3b0n1F%2BbACM0sO0gvmmXAZNgbeOIpsUAclRw%3D%3D" -O "Reviews.csv.zip" -c

# remove '#' sign to unzip the dataset
"""import zipfile
data=zipfile.ZipFile("Reviews.csv.zip")
data.extractall()""

In [2]:
#loading the amazon dataset
dataset=pd.read_csv("Reviews.csv")

In [3]:
print(dataset.shape)
dataset.head()


(568454, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


> # preprocessing the data

In [4]:
# sorting the value
dataset.sort_values(by='Id',inplace=True )
#finding the dublicate values using 'df.dublicated'
dataset[dataset.duplicated(subset={'ProfileName','HelpfulnessNumerator','HelpfulnessDenominator','Score','Time'})].shape
#alternate way to drop dublicate values
dataset_no_dup=dataset.drop_duplicates(subset={'ProfileName','Score','Time','Summary'},keep='first')
print(f"before {dataset.shape}")
print(f"after removing duplicate values-->shape = {dataset_no_dup.shape}")
# %age of no. of review reamin in data set
(dataset_no_dup.size/dataset.size)*100

# removing reviews where "HelpfulnessNumerator>HelpfulnessDenominator"
dataset_no_dup=dataset_no_dup[dataset_no_dup['HelpfulnessNumerator']<=dataset_no_dup['HelpfulnessDenominator']]


before (568454, 10)
after removing duplicate values-->shape = (393141, 10)


In [5]:
# taking reviews whose score is not equal to 3
filtered_dataset=dataset_no_dup[dataset_no_dup['Score']!=3]
filtered_dataset.shape
#creating a function to filter the reviews (if score>3 --> positive , if score<3 --> negative)
def partition(x):
    if x>3:
        return 'positive'
    else:
        return 'negative'

score=filtered_dataset['Score']
pos_neg=score.map(partition)
filtered_dataset['Score']=pos_neg
print(filtered_dataset.shape)
filtered_dataset.head()



(363393, 10)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [25]:
#changing the format of timestamp to ('%Y-%m-%d %H:%M:%S'
import time
import datetime
time=[]
for timestamp in filtered_dataset['Time']:
    t=datetime.datetime.fromtimestamp(timestamp).strftime(('%Y-%m-%d %H:%M:%S'))
    time.append(t)
filtered_dataset['time']=time   

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:
# sort by time
filtered_dataset.sort_values(by='time',inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
import re
import nltk
from nltk.corpus import stopwords

sno = nltk.stem.SnowballStemmer('english')#snowball stemmer

stop=set(stopwords.words('english')) #set of stopwords

#clean html tags
def cleanhtml(sent):
    pattern=re.compile(r'<.*?>')
    cleansent=re.sub(pattern," ",sent)
    return cleansent

#cleean punctuation
def cleanpunc(word):
    clean_punc=re.sub(r'[?|!|\'|"|#]',' ',word)
    clean_punc=re.sub(r'[.|,|)|(|\|/]',' ',clean_punc)
    return clean_punc

In [7]:
import string

x=0 # number of reviews you want to clean/pre-processed
positive_words=[]
negative_words=[]
str1=''
final_sent=[] # storing the list of final pre-processed sentences
i=0

for sent in filtered_dataset["Text"]:
    sent=cleanhtml(sent) #removing html tags
    filtered_sentence=[]
    for w in sent.split():
        for clean_words in cleanpunc(w).split():
            if((len(clean_words)>2) & (clean_words.isalpha())):
                if (clean_words.lower() not in stop):
                    s=(sno.stem(clean_words.lower()))
                    filtered_sentence.append(s)
                    if (filtered_dataset['Score'].values[i]=='positive'):
                        positive_words.append(s)
                    if (filtered_dataset['Score'].values[i]=='negative'):
                        negative_words.append(s)
                else:
                    continue
            else:
                continue
    str1=" ".join(filtered_sentence) #str of all the cleaned words
    final_sent.append(str1) # appending cleaned words to sentence
    i=i+1
          

In [8]:
print(len(final_sent))
print("\nBefore cleaning :\n",filtered_dataset["Text"][10])
print(f"\nAfter cleaning :\n {final_sent[10]}")



363393

Before cleaning :
 I don't know if it's the cactus or the tequila or just the unique combination of ingredients, but the flavour of this hot sauce makes it one of a kind!  We picked up a bottle once on a trip we were on and brought it back home with us and were totally blown away!  When we realized that we simply couldn't find it anywhere in our city we were bummed.<br /><br />Now, because of the magic of the internet, we have a case of the sauce and are ecstatic because of it.<br /><br />If you love hot sauce..I mean really love hot sauce, but don't want a sauce that tastelessly burns your throat, grab a bottle of Tequila Picante Gourmet de Inclan.  Just realize that once you taste it, you will never want to use any other sauce.<br /><br />Thank you for the personal, incredible service!

After cleaning :
 know cactus tequila uniqu combin ingredi flavour hot sauc make one kind pick bottl trip brought back home total blown away realiz simpli find anywher citi bum magic internet 

In [9]:
filtered_dataset['cleaned_text']=final_sent
print(filtered_dataset.shape)
filtered_dataset.head()

(363393, 11)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,cleaned_text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,bought sever vital can dog food product found ...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,product arriv label jumbo salt peanut peanut a...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,confect around centuri light pillowi citrus ge...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,look secret ingredi robitussin believ found go...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...,great taffi great price wide assort yummi taff...


## 1. Bag of word

In [10]:
# one gram-BOW from "sklean.feature_extraction.text.CountVectorizer"
count_vect=CountVectorizer()
bow_cleaned_text=count_vect.fit_transform(filtered_dataset['cleaned_text'])


In [11]:
# two gram_BOW 
count_vect_gram=CountVectorizer(ngram_range=(1,2))
bow_cleaned_text_2gram=count_vect_gram.fit_transform(filtered_dataset['cleaned_text'])

In [12]:
print("bow_cleaned_text",bow_cleaned_text.shape)
print("bow_cleaned_text_2gram",bow_cleaned_text_2gram.shape)

bow_cleaned_text (363393, 70047)
bow_cleaned_text_2gram (363393, 2881737)


###  PCA visualisation of BOW- 1 gram

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler(with_mean=False)
a=sc.fit_transform(bow_cleaned_text[:4000])

In [ ]:
a.shape

In [ ]:
a=a.todense()

In [ ]:
from sklearn.manifold import TSNE
model = TSNE(n_components=2, random_state=0, perplexity = 30, n_iter = 5000)
# configuring the parameteres
# the number of components = 2
# default perplexity = 30
# default learning rate = 200
# default Maximum number of iterations for the optimization = 1000

tsne_data = model.fit_transform(a)


# creating a new data frame which help us in ploting the result data
tsne_data = np.vstack((tsne_data.T, filtered_dataset['Text'][:4000])).T
tsne_df = pd.DataFrame(data=tsne_data, columns=("dim1", "dim2", "score"))

# Ploting the result of tsne
sns.FacetGrid(tsne_df, hue="score", size=6).map(plt.scatter, 'dim1', 'dim2').add_legend()
plt.title("TSNE for Bag of Words")
plt.show()

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt
# Ploting the result of tsne
sns.FacetGrid(tsne_df, hue="score", height=6).map(plt.scatter, 'dim1', 'dim2').add_legend()
plt.title("TSNE for Bag of Words")
plt.show()

## 2. TF-IDF

In [35]:
# tf-idf "from sklearn.feature_extraction.text.TfidfVectorizer"
tf_idf=TfidfVectorizer()
tf_idf_cleaned_text=tf_idf.fit_transform(filtered_dataset['cleaned_text'])

In [36]:
tf_idf_cleaned_text.shape

(363393, 70047)

In [28]:
# tf-idf 2-gram --> increasing no. of gram can increase the no. of dimension drastically
tf_idf_2gram = TfidfVectorizer(ngram_range=(1,2))
tf_idf_cleaned_text_2gram = tf_idf_2gram.fit_transform(filtered_dataset['cleaned_text'].values)
print("the type of count vectorizer ",type(tf_idf_cleaned_text_2gram))
print("the shape of out text TFIDF vectorizer ",tf_idf_cleaned_text_2gram.get_shape())
print("the number of unique words including both unigrams and bigrams ", tf_idf_cleaned_text_2gram.get_shape()[1])

the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text TFIDF vectorizer  (363393, 2881737)
the number of unique words including both unigrams and bigrams  2881737


## 3. avg word2vec

In [ ]:
"""# understanding w2v using goolgle trained model of 300 dimension
# w2v lib from "gensim.models import KeyedVectors"
from gensim.models import KeyedVectors
google_w2v=KeyedVectors.load_word2vec_format("../input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin.gz",encoding='utf8',binary=True)
"""

In [ ]:
"""google_w2v.distance('woman','queen')"""

In [15]:
# converting our text-->vector using w2v with 50-dim
# more the dimension of each word = better the semantic of word
# using lib from "gensim.models.Word2Vec"
# to run w2v we need list of list of the words as w2v covert each world into number of dim

list_of_sent=[]
for sent in filtered_dataset['cleaned_text'].values:
    list_of_sent.append((str(sent)).split())
w2v_model=Word2Vec(list_of_sent,min_count=5,size=50)
# vocablary of w2v model of amazon dataset
vocab=w2v_model.wv.vocab
len(vocab)

21817

In [ ]:
# understanding w2v on amzon fine food reviews dataset
w2v_model.similar_by_word('women')

In [ ]:
# w2v representaion of word "women" in 50-dim
w2v_model.wv['women']

In [16]:
'''
-->procedure to make avg w2v of each reviews
    1. find the w2v of each word
    2. sum-up w2v of each word in a sentence
    3. divide the total w2v of sentence by total no. of words in the sentence
'''

# average Word2Vec
# compute average word2vec for each review.
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list

for sent in list_of_sent[:20000]: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in vocab:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors.append(sent_vec)

print(len(sent_vectors))
print(len(sent_vectors[0]))

20000
50


In [17]:
sent_vectors[0] #avg w2v of first sentences

array([ 0.01603449,  0.49089715, -0.10807862, -2.10303584,  0.69066381,
       -1.13479463, -0.86968518,  0.06382391, -0.08572455, -0.68432397,
        0.8616731 , -0.71690129, -0.88386354,  0.00310096, -0.19346318,
       -0.49344944,  0.30754587, -0.56316829, -0.0867224 , -1.29980897,
        0.61145266,  0.77000769,  0.42954946, -0.28112701, -0.48668469,
        0.3200143 , -0.33681095,  0.27818683,  0.4912024 ,  0.5521246 ,
       -0.46394361,  0.36881682, -0.73963399,  0.33445731, -0.14917238,
       -0.32202016, -0.12751943, -0.53720554,  0.16698155, -0.0551241 ,
        0.50743894, -0.44015141,  1.16529965, -1.06775678, -0.26140797,
       -0.01735742,  0.49543894, -0.33265766,  1.18862955,  0.36698836])

## avg  TF-IDF W2V 

In [38]:
# tfidf words/col names
tfidf_feat = tf_idf.get_feature_names()

In [41]:
# TF-IDF weighted Word2Vec

# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in list_of_sent[:10000]: # for each review/sentence 
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in vocab:
            vec = w2v_model.wv[word]
            # obtain the tf_idfidf of a word in a sentence/review
            tf_idf = tf_idf_cleaned_text[row, tfidf_feat.index(word)]
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1

In [42]:
len(tfidf_sent_vectors)

10000

## K-NN on amazon fine food dataset using different hyperparameter

In [43]:
# function of K-NN with different hyperparamter
def kfold_knn_1(X,y):

    x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

    # using k-fold to find best k-value in KNN
    acc=[]
    for k in range(1,15,2):
        clf=KNeighborsClassifier(n_neighbors=k , weights='uniform')
        cv_scores=cross_val_score(clf,x_train,y_train,cv=10,scoring='accuracy')
        acc.append(cv_scores.mean()*float(100))

    max_acc=acc[acc.index(max(acc))] # maximum accuarcy
    k=acc.index(max(acc))+1 # best k-value
    print("\n**************************k-fold knn (n_neighbors=k , weights='uniform') **********************************")
    print('**************** using k-fold to find best K-value **********************')
    print(f'best accuracy is {max_acc} on cv datatset using 10 fold at k-value {k}')

    #using best k-value to find generalistion value
    clf=KNeighborsClassifier(n_neighbors=k)
    clf.fit(x_train,y_train)
    y_pred=clf.predict(x_test)
    accuracy=accuracy_score(y_test,y_pred)
    print(f'genearalisation accuracy on best k-value at k = {k} is accuacy = {accuracy}')
    
#----------------------------------------------------------------------------------------------------------------------------    
def kfold_knn_2(X,y):

    x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

    # using k-fold to find best k-value in KNN
    acc=[]
    for k in range(1,15,2):
        clf=KNeighborsClassifier(n_neighbors=k , weights='distance')
        cv_scores=cross_val_score(clf,x_train,y_train,cv=10,scoring='accuracy')
        acc.append(cv_scores.mean()*float(100))

    max_acc=acc[acc.index(max(acc))] # maximum accuarcy
    k=acc.index(max(acc))+1 # best k-value
    print("\n**************************k-fold knn (clf=KNeighborsClassifier(n_neighbors=k , weights='distance')**********************************")
    print('**************** using k-fold to find best K-value **********************')
    print(f'best accuracy is {max_acc} on cv datatset using 10 fold at k-value {k}')

    #using best k-value to find generalistion value
    clf=KNeighborsClassifier(n_neighbors=k)
    clf.fit(x_train,y_train)
    y_pred=clf.predict(x_test)
    accuracy=accuracy_score(y_test,y_pred)
    print(f'genearalisation accuracy on best k-value at k = {k} is accuacy = {accuracy}')
    
#----------------------------------------------------------------------------------------------------------------------------    

def kfold_knn_3(X,y):

    x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

    # using k-fold to find best k-value in KNN
    acc=[]
    for k in range(1,15,2):
        clf=KNeighborsClassifier(n_neighbors=k , weights='uniform' , algorithm='brute')
        cv_scores=cross_val_score(clf,x_train,y_train,cv=10,scoring='accuracy')
        acc.append(cv_scores.mean()*float(100))

    max_acc=acc[acc.index(max(acc))] # maximum accuarcy
    k=acc.index(max(acc))+1 # best k-value
    print("\n**************************k-fold knn (n_neighbors=k , weights='uniform' , algorithm='brute') **********************************")
    print('**************** using k-fold to find best K-value **********************')
    print(f'best accuracy is {max_acc} on cv datatset using 10 fold at k-value {k}')

    #using best k-value to find generalistion value
    clf=KNeighborsClassifier(n_neighbors=k)
    clf.fit(x_train,y_train)
    y_pred=clf.predict(x_test)
    accuracy=accuracy_score(y_test,y_pred)
    print(f'genearalisation accuracy on best k-value at k = {k} is accuacy = {accuracy}')


#----------------------------------------------------------------------------------------------------------------------------    

def kfold_knn_4(X,y):

    x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

    # using k-fold to find best k-value in KNN
    acc=[]
    for k in range(1,15,2):
        clf=KNeighborsClassifier(n_neighbors=k , weights='distance' , algorithm='brute')
        cv_scores=cross_val_score(clf,x_train,y_train,cv=10,scoring='accuracy')
        acc.append(cv_scores.mean()*float(100))

    max_acc=acc[acc.index(max(acc))] # maximum accuarcy
    k=acc.index(max(acc))+1 # best k-value
    print("\n**************************k-fold knn (n_neighbors=k , weights='distance' , algorithm='brute') **********************************")
    print('**************** using k-fold to find best K-value **********************')
    print(f'best accuracy is {max_acc} on cv datatset using 10 fold at k-value {k}')

    #using best k-value to find generalistion value
    clf=KNeighborsClassifier(n_neighbors=k)
    clf.fit(x_train,y_train)
    y_pred=clf.predict(x_test)
    accuracy=accuracy_score(y_test,y_pred)
    print(f'genearalisation accuracy on best k-value at k = {k} is accuacy = {accuracy}')
#----------------------------------------------------------------------------------------------------------------------------    

def kfold_knn_5(X,y):

    x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

    # using k-fold to find best k-value in KNN
    acc=[]
    for k in range(1,15,2):
        clf=KNeighborsClassifier(n_neighbors=k , weights='uniform' , algorithm='kd_tree' )
        cv_scores=cross_val_score(clf,x_train,y_train,cv=10,scoring='accuracy')
        acc.append(cv_scores.mean()*float(100))

    max_acc=acc[acc.index(max(acc))] # maximum accuarcy
    k=acc.index(max(acc))+1 # best k-value
    print("\n**************************k-fold knn (n_neighbors=k , weights='uniform' , algorithm='kd_tree' ) **********************************")
    print('**************** using k-fold to find best K-value **********************')
    print(f'best accuracy is {max_acc} on cv datatset using 10 fold at k-value {k}')

    #using best k-value to find generalistion value
    clf=KNeighborsClassifier(n_neighbors=k)
    clf.fit(x_train,y_train)
    y_pred=clf.predict(x_test)
    accuracy=accuracy_score(y_test,y_pred)
    print(f'genearalisation accuracy on best k-value at k = {k} is accuacy = {accuracy}')

#----------------------------------------------------------------------------------------------------------------------------    

def kfold_knn_6(X,y):

    x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

    # using k-fold to find best k-value in KNN
    acc=[]
    for k in range(1,15,2):
        clf=KNeighborsClassifier(n_neighbors=k , weights='distance' , algorithm='kd_tree' )
        cv_scores=cross_val_score(clf,x_train,y_train,cv=10,scoring='accuracy')
        acc.append(cv_scores.mean()*float(100))

    max_acc=acc[acc.index(max(acc))] # maximum accuarcy
    k=acc.index(max(acc))+1 # best k-value
    print("\n**************************k-fold knn (n_neighbors=k , weights='distance' , algorithm='kd_tree' ) **********************************")
    print('**************** using k-fold to find best K-value **********************')
    print(f'best accuracy is {max_acc} on cv datatset using 10 fold at k-value {k}')

    #using best k-value to find generalistion value
    clf=KNeighborsClassifier(n_neighbors=k)
    clf.fit(x_train,y_train)
    y_pred=clf.predict(x_test)
    accuracy=accuracy_score(y_test,y_pred)
    print(f'genearalisation accuracy on best k-value at k = {k} is accuacy = {accuracy}')


    

> ## Data splitiing for training and checking accuracy on (bow , tf-idf , avg w2v , avg tf-idf w2v)

## *  K-NN ON BOW 1-gram


In [45]:
#shaping datset for 'X' and 'y' for classification model
X=bow_cleaned_text[:50000]
y=filtered_dataset['Score'][:50000]
#calling function of KNN with different parameter
kfold_knn_1(X,y)
kfold_knn_2(X,y)
kfold_knn_3(X,y)
kfold_knn_4(X,y)



**************************k-fold knn (n_neighbors=k , weights='uniform') **********************************
**************** using k-fold to find best K-value **********************
best accuracy is 88.78857351113719 on cv datatset using 10 fold at k-value 7
genearalisation accuracy on best k-value at k = 7 is accuacy = 0.8841333333333333

**************************k-fold knn (clf=KNeighborsClassifier(n_neighbors=k , weights='distance')**********************************
**************** using k-fold to find best K-value **********************
best accuracy is 88.74857187661812 on cv datatset using 10 fold at k-value 7
genearalisation accuracy on best k-value at k = 7 is accuacy = 0.8841333333333333

**************************k-fold knn (n_neighbors=k , weights='uniform' , algorithm='brute') **********************************
**************** using k-fold to find best K-value **********************
best accuracy is 88.78857351113719 on cv datatset using 10 fold at k-value 7
genearalisa

In [46]:
#shaping datset for 'X' and 'y' for classification model
x_train=bow_cleaned_text[:40000]
y_train=filtered_dataset['Score'][:40000]
x_test=bow_cleaned_text[40000:50000]
y_test=filtered_dataset['Score'][40000:50000]

In [48]:
    # function of K-NN with different hyperparamter
    #def kfold_knn_1(X,y):

    #x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

    # using k-fold to find best k-value in KNN
    acc=[]
    for k in range(1,15,2):
        clf=KNeighborsClassifier(n_neighbors=k , weights='uniform')
        cv_scores=cross_val_score(clf,x_train,y_train,cv=10,scoring='accuracy')
        acc.append(cv_scores.mean()*float(100))

    max_acc=acc[acc.index(max(acc))] # maximum accuarcy
    k=acc.index(max(acc))+1 # best k-value
    print("\n**************************k-fold knn (n_neighbors=k , weights='uniform') **********************************")
    print('**************** using k-fold to find best K-value **********************')
    print(f'best accuracy is {max_acc} on cv datatset using 10 fold at k-value {k}')

    #using best k-value to find generalistion value
    clf=KNeighborsClassifier(n_neighbors=k)
    clf.fit(x_train,y_train)
    y_pred=clf.predict(x_test)
    accuracy=accuracy_score(y_test,y_pred)
    print(f'genearalisation accuracy on best k-value at k = {k} is accuacy = {accuracy}')
    


**************************k-fold knn (n_neighbors=k , weights='uniform') **********************************
**************** using k-fold to find best K-value **********************
best accuracy is 88.97500048734379 on cv datatset using 10 fold at k-value 7
genearalisation accuracy on best k-value at k = 7 is accuacy = 0.8746


## * K-NN on TF-IDF 1-gram


In [ ]:
X1=tf_idf_cleaned_text[:20000]
y1=filtered_dataset['Score'][:20000]
#calling function of KNN with different parameter
kfold_knn_4(X1,y1)

## * K-NN on avg W2V


In [ ]:
X2=sent_vectors[:20000]
y2=filtered_dataset['Score'][20000]
#calling function of KNN with different parameter
kfold_knn_4(X2,y2)
kfold_knn_5(X2,y2)

## * K-NN on avg TF-IDF W2V

In [ ]:
X3=tfidf_sent_vectors[:20000]
y3=filtered_dataset['Score'][:20000]
#calling function of KNN with different parameter
kfold_knn_4(X3,y3)
kfold_knn_5(X3,y3)

# Time based splliting and apply K-NN

In [ ]:
#shaping datset for 'X' and 'y' for classification model
x_train=bow_cleaned_text[:40000]
y_train=filtered_dataset['Score'][:40000]
x_test=bow_cleaned_text[40000:50000]
y_test=filtered_dataset['Score'][40000:50000]


# function of K-NN for time based splitig dataset
def kfold_knn_1(x_train,y_train,x_test,y_test):

    

    # using k-fold to find best k-value in KNN
    acc=[]
    for k in range(1,15,2):
        clf=KNeighborsClassifier(n_neighbors=k , weights='uniform')
        cv_scores=cross_val_score(clf,x_train,y_train,cv=10,scoring='accuracy')
        acc.append(cv_scores.mean()*float(100))

    max_acc=acc[acc.index(max(acc))] # maximum accuarcy
    k=acc.index(max(acc))+1 # best k-value
    print("\n**************************k-fold knn (n_neighbors=k , weights='uniform') **********************************")
    print('**************** using k-fold to find best K-value **********************')
    print(f'best accuracy is {max_acc} on cv datatset using 10 fold at k-value {k}')

    #using best k-value to find generalistion value
    clf=KNeighborsClassifier(n_neighbors=k)
    clf.fit(x_train,y_train)
    y_pred=clf.predict(x_test)
    accuracy=accuracy_score(y_test,y_pred)
    print(f'genearalisation accuracy on best k-value at k = {k} is accuacy = {accuracy}')